# Groupe 11 - TP 2 - Rapport

Membres du groupe:
- Oussama Khaloui --- khao1201
- Caroline Wang --- wanc1101

#### Les imports

In [18]:
import pandas as pd

#### Prétraitements

In [19]:
# Lecture des données
orders_distance_stores_softmax = pd.read_csv("donnees\orders_distance_stores_softmax.csv")
orders_products_prior_specials = pd.read_csv("donnees\order_products__prior_specials.csv")

In [20]:
orders_distance_stores_softmax = orders_distance_stores_softmax.drop(columns=['Unnamed: 0'])
orders_distance_stores_softmax.head()

,user_id,store_id,distance,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,210,0,2.304404,1438665,prior,9,1,18,3.0
1,210,0,2.304404,2850206,prior,36,0,7,2.0
2,210,0,2.304404,2406913,prior,42,0,8,3.0
3,210,0,2.304404,1155933,prior,43,2,23,2.0
4,210,0,2.304404,271697,prior,56,5,17,3.0


In [21]:
orders_products_prior_specials = orders_products_prior_specials.drop(columns=['Unnamed: 0'])
orders_products_prior_specials.head()

,order_id,product_id,add_to_cart_order,reordered,special
0,15,19660,1,1,15
1,15,21195,2,1,0
2,15,7461,3,1,50
3,15,2996,4,1,0
4,15,32463,5,1,0


In [22]:
orders = pd.merge(orders_distance_stores_softmax, orders_products_prior_specials, on='order_id', how='inner')
orders.dropna(inplace=True)
print(len(orders))

1158441


In [23]:
# Réduction des données à 5%
orders = orders.sample(frac=0.05, random_state=42)
list(orders.columns)

['user_id',
 'store_id',
 'distance',
 'order_id',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order',
 'product_id',
 'add_to_cart_order',
 'reordered',
 'special']

In [24]:
from sklearn.model_selection import train_test_split

# Division des données en ensemble d'entraînement et ensemble de test
X = orders.drop(columns=['reordered'])
#X = pd.get_dummies(X, columns=['eval_set'])
X = orders.drop(columns=['eval_set'])
y = orders['reordered']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
from sklearn.discriminant_analysis import StandardScaler

standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.fit_transform(X_test)

#### 2. Choix du modèle de prédiction

In [26]:
# Recherche d'hyperpatamètres en fonction du noyau
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit

def hyper_parameters_search(model, kernel, X_train, y_train):
        if kernel == 'linear':
            C_range = [1,10,100,1000]
            param_grid = dict(C=C_range)
            cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
            grid = GridSearchCV(model, param_grid=param_grid, cv=cv, scoring='accuracy')
            grid.fit(X_train, y_train)
            print(grid.best_score_, " pour les hyperparamètres suivants, noyau RBF : ", grid.best_params_)
            return grid.best_estimator_, grid.best_params_
        elif kernel == 'rbf':
            C_range = [1,10,100,1000]
            gamma_range = [1,0.1,0.001,0.0001]
            param_grid = dict(gamma=gamma_range, C=C_range)
            cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
            grid = GridSearchCV(model, param_grid=param_grid, cv=cv, scoring='accuracy')
            grid.fit(X_train, y_train)
            print(grid.best_score_, " pour les hyperparamètres suivants, noyau RBF : ", grid.best_params_)
            return grid.best_estimator_, grid.best_params_
        elif kernel == 'sigmoid':
            coef0_range = [-5, -3, -1, -0.1, 0.0, 0.1, 1]
            param_grid = dict(coef0=coef0_range)
            cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
            grid = GridSearchCV(model, param_grid=param_grid, cv=cv, scoring='accuracy')
            grid.fit(X_train, y_train)
            print(grid.best_score_, " pour les hyperparamètres suivants, noyau sigmoïde : ", grid.best_params_)
            return grid.best_estimator_, grid.best_params_
        elif kernel == 'poly':
            coef0_range = [-5, -3, -1, -0.1, 0.0, 0.1, 1]
            degree_range = [2, 3, 4, 5, 6, 7]
            param_grid = dict(coef0=coef0_range, degree = degree_range)
            cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
            grid = GridSearchCV(model, param_grid=param_grid, cv=cv, scoring='accuracy')
            grid.fit(X_train, y_train)
            print(grid.best_score_, " pour les hyperparamètres suivants, noyau polynomial : ", grid.best_params_)
            return grid.best_estimator_, grid.best_params_
        print("Erreur recherche hyper-paramètres")
        return model     

In [27]:
import joblib
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, f1_score

kernel_values = ['linear', 'rbf', 'sigmoid', 'poly']
for kernel in kernel_values :
    print(f'----- Modèle de noyau {kernel} -----')
    
    svm_model = SVC(kernel=kernel)
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    
    # Évaluation du modèle    
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f'Matrice de confusion :')
    print(conf_matrix)
    
    f1_svm = f1_score(y_test, y_pred)
    print(f'Score F1 : {f1_svm}')
    
    # Recherche des hyperparamètres
    best_svm_model, best_svm_param = hyper_parameters_search(svm_model, kernel, X_train=X_train, y_train=y_train)
    best_svm_model.fit(X_train, y_train)
    best_y_pred = best_svm_model.predict(X_test)

    # Évaluation du meilleur
    best_conf_matrix = confusion_matrix(y_test, best_y_pred)
    print(f'Matrice de confusion')
    print(best_conf_matrix)
    
    best_f1_svm = f1_score(y_test, best_y_pred)
    print(f'Score F1 : {best_f1_svm}')
    
    # Sauvegarde du modèle et des hyperparamètres dans un fichier
    joblib.dump(best_svm_model, f'modele_svm_{kernel}.pkl')
    joblib.dump(best_svm_param, f'hyperparametres_svm_{kernel}.pkl')

----- Modèle de noyau linear -----
Matrice de confusion :
[[2481    0]
 [   0 9104]]


Score F1 : 1.0
1.0  pour les hyperparamètres suivants, noyau RBF :  {'C': 1}
Matrice de confusion
[[2481    0]
 [   0 9104]]
Score F1 : 1.0
----- Modèle de noyau rbf -----
Matrice de confusion :
[[2481    0]
 [   0 9104]]
Score F1 : 1.0


In [17]:
# Chargement des modèle et des hyperparamètres à partir des fichiers sauvegardés
kernel_values = ['linear', 'rbf', 'sigmoid', 'poly']
for kernel in kernel_values :
    modele_svm = joblib.load(f'modele_svm_{kernel}.pkl')
    hyperparametres_svm = joblib.load(f'hyperparametres_svm_{kernel}.pkl')
    print(f'Meilleurs paramètres du modèle SVM avec noyau de type {kernel} : {hyperparametres_svm}')

Meilleurs paramètres du modèle SVM avec noyau de type linear : {'C': 1}


#### 3. Conclusion

(a)

(b) Les hyperparamètres utilisés pour le modèle SVM sont les suivants :

- Pour le noyau linear :
    - C : la valeur de pénalité du terme d'erreur.

- Pour le noyau rbf :
    - C : la valeur de pénalité du terme d'erreur.
    - gamma : la valeur du coefficient du noyau RBF.

- Pour le noyau sigmoid :
    - coef0 : la valeur de l'ordonnée à l'origine dans le noyau sigmoïde.

- Pour le noyau poly :
    - coef0 : la valeur de l'ordonnée à l'origine dans le noyau polynomial.
    - degree : la valeur du degré du noyau polynomial.

(c)

(d) Le score F1 est très bon, il est proche de 1. Cela signifie que le modèle a bien été entrainé et que ses capacités de prédiction sont satisfaisantes.